In [1]:
import os  # os 모듈 운영체제와 상호 작용할 수 있는 기능을 제공
import torch # PyTorch 라이브러리로, 주로 딥러닝과 머신러닝 모델을 구축, 학습, 테스트하는 데 사용
from datasets import load_dataset  # 데이터셋을 쉽게 불러오고 처리할 수 있는 기능을 제공
from transformers import (
    AutoModelForCausalLM, # 인과적 언어 추론(예: GPT)을 위한 모델을 자동으로 불러오는 클래스
    AutoTokenizer, # 입력 문장을 토큰 단위로 자동으로 잘라주는 역할 
    BitsAndBytesConfig, # 모델 구성
    HfArgumentParser,  # 파라미터 파싱
    TrainingArguments,  # 훈련 설정
    pipeline,  # 파이프라인 설정
    logging,  #로깅을 위한 클래스
)

# 모델 튜닝을 위한 라이브러리
from peft import LoraConfig, PeftModel  
from trl import SFTTrainer

2024-06-21 13:39:51.550902: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-21 13:39:51.608679: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-21 13:39:52.696447: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
No ROCm runtime is found, using ROCM_HOME='/opt/rocm-6.1.2'


In [2]:
torch.cuda.empty_cache()

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0,1,3"  # Set the GPUs 2 and 3 to use

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 3


In [3]:
# Convert llama weight to llamion weight
#model_dir = "../llama/llama-2-7b-chat-convert-weight"
model_dir = "vaiv/Gem2-Llamion-14B-Base"
# REFT model
new_model = "llamion_peft"

In [4]:
# Model
base_model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map='auto'
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

/home/dxlab/anaconda3/envs/obqa/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [5]:
# Tokenizer
base_tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
base_tokenizer.pad_token = base_tokenizer.eos_token
base_tokenizer.padding_side = "right"

/home/dxlab/anaconda3/envs/obqa/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
import torch.nn as nn
for param in base_model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

base_model.gradient_checkpointing_enable()  # reduce number of stored activations
base_model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
base_model.lm_head = CastOutputToFloat(base_model.lm_head)

In [7]:
# for param in base_model.parameters():
#     print(param.requires_grad, sep="   ")

In [8]:
# Data set
# https://huggingface.co/datasets/mlabonne/guanaco-llama2-1k
data_name = "vaiv/ko-rag-preference"
training_data = load_dataset(data_name, split="validation")

print(type(training_data))
# check the data
print(training_data.shape)
# #11 is a QA sample in English
print(training_data)

<class 'datasets.arrow_dataset.Dataset'>
(50, 4)
Dataset({
    features: ['question', 'context', 'chosen', 'rejected'],
    num_rows: 50
})


In [27]:
import random
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
import torch.nn.functional as F
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def set_seed(seed):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

class PolyEncoder(torch.nn.Module):
    def __init__(self, bert_model_name='klue/bert-base', poly_m=16):
        super(PolyEncoder, self).__init__()
        self.poly_m = poly_m
        self.bert_model = BertModel.from_pretrained(bert_model_name)
        self.poly_code_embeddings = torch.nn.Embedding(poly_m, self.bert_model.config.hidden_size)
        
    def forward(self, context_input_ids, context_attention_mask, question_input_ids, question_attention_mask):
        # Encode the question
        question_outputs = self.bert_model(input_ids=question_input_ids, attention_mask=question_attention_mask)
        question_cls_embeddings = question_outputs.last_hidden_state[:, 0, :]  # CLS token

        # Encode the context
        context_outputs = self.bert_model(input_ids=context_input_ids, attention_mask=context_attention_mask)
        context_hidden_states = context_outputs.last_hidden_state

        # Poly codes
        poly_codes = self.poly_code_embeddings.weight.unsqueeze(0).expand(context_hidden_states.size(0), -1, -1)

        # Context and poly code interactions
        attention_weights = F.softmax(torch.einsum('bmd,bnd->bmn', context_hidden_states, poly_codes), dim=-1)
        poly_context_embeddings = torch.einsum('bmn,bmd->bnd', attention_weights, context_hidden_states)

        # Question and poly context interactions
        scores = torch.einsum('bnd,bmd->bnm', poly_context_embeddings, question_cls_embeddings.unsqueeze(1).expand(-1, self.poly_m, -1))

        # Aggregate scores over poly_m dimension
        scores = scores.max(dim=1).values

        return scores

def get_top_n_relevant_sentences(context, question, tokenizer, model, top_n):
    context_sentences = sent_tokenize(context)  # NLTK를 사용하여 문장 분할
    print(f"Context sentences: {context_sentences}")  # 문장 분할 결과 출력

    context_inputs = tokenizer(context_sentences, padding=True, truncation=True, return_tensors='pt')
    question_inputs = tokenizer(question, return_tensors='pt')

    # 입력 텐서의 길이 확인
    print(f"Context input IDs shape: {context_inputs['input_ids'].shape}")
    print(f"Question input IDs shape: {question_inputs['input_ids'].shape}")

    with torch.no_grad():
        scores = model(context_inputs['input_ids'], context_inputs['attention_mask'], 
                       question_inputs['input_ids'].expand(len(context_sentences), -1),
                       question_inputs['attention_mask'].expand(len(context_sentences), -1))

    # 디버깅용 출력
    print(f"Number of context sentences: {len(context_sentences)}")
    print(f"Scores shape: {scores.shape}")
    score_rows, score_cols = scores.shape

    print(f"Scores: {scores}")

    scores_index = scores[:, 0].tolist()
    print(scores_index)
    indexed_dict = {idx: value for idx, value in enumerate(scores_index)}
    sorted_dict = dict(sorted(indexed_dict.items(), key=lambda item: item[1], reverse=True))
    sorted_data = sorted(sorted_dict.items(), key=lambda item: item[1], reverse=True)
    print(sorted_data)
    top_n_keys = list(sorted_dict.keys())[:top_n]
    unique_values = set()
    top_keys = []

    for key, value in sorted_data:
        if value not in unique_values:
            unique_values.add(value)
            top_keys.append(key)
        if len(top_keys) == top_n:
            break

    print(f"Top {top_n} sentence indices: {top_keys}")
    top_n_sentences = [context_sentences[idx] for idx in top_keys]
    return top_n_sentences

# 모델 및 토크나이저 로드를 전역 변수로 설정
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')
model = PolyEncoder(bert_model_name='klue/bert-base')

# 예제 실행 함수
def run_example(context, question):

    top_n_sentences = get_top_n_relevant_sentences(context, question, tokenizer, model, top_n=5)
    sentences = ""
    for sentence in top_n_sentences:
        sentences+=sentence
    print(sentences)
    return sentences


[nltk_data] Downloading package punkt to /home/dxlab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/home/dxlab/anaconda3/envs/obqa/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
from datasets import Dataset
dataset_dic = {
    'text' :[]
}
for dataset in training_data:
    format_data = f'<|begin_of_text|><|start_header_id|>context<|end_header_id|>\n\n{dataset["context"]}<|eot_id|><|start_header_id|>question<|end_header_id|>\n\n{dataset["question"]}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{dataset["chosen"]}<|eot_id|><|end_of_text|>'
    dataset_dic['text'].append(format_data)
for i in range(2):
    for dataset in training_data:
        torch.cuda.empty_cache()
        context = run_example(dataset['context'], dataset['question'])
        format_data = f'<|begin_of_text|><|start_header_id|>context<|end_header_id|>\n\n{context}<|eot_id|><|start_header_id|>question<|end_header_id|>\n\n{dataset["question"]}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{dataset["chosen"]}<|eot_id|><|end_of_text|>'
        dataset_dic['text'].append(format_data)
dataset_training_data = Dataset.from_dict(dataset_dic)
print(dataset_training_data)

강한 자외선과 더운 날씨에 외출을 가급적 삼가를 해야 하며 몸으로 빠져나가는 수분을 이온음료, 물 등을 수시로 섭취하여 보충해줘야 합니다.또한, 규칙적으로 물을 마시고 몸에 수분이 부족하지 않도록 주의하는 것도 좋습니다.일사병은 대체적으로 염분과 수분의 부족으로 나타나는 경우가 많으므로 규칙적인 수분 섭취를 통해 예방할 수 있습니다.따라서 부득이하게 야외에 나갈 때는 챙이 넓은 모자를 쓰고 헐렁한 옷을 입는 것이 좋습니다.일시적으로 일사병의 증상이 나타나는 경우 서늘한 곳에서 휴식을 취하고 수분을 섭취하면 호전되는 경우가 많습니다.
8.1.[4] 입장료는 무료이지만 사전 예약을 권장합니다 .[5] 7.[1] 시카고에는 세계적으로 유명한 예술 박물관, 미술관, 공원, 호수, 대학, 스포츠 경기장 등이 있습니다.
그레이엄의 수제자인 워런 버핏도 이 같은 행동주의 전략을 적극 활용했다.[2] 적극적으로 행사해 그 차이를 빠르게 줄이는 전략이다.이후 행동주의는 1980~1990년대 ‘기업 사냥꾼 시대’를 거쳐 2000년대 들어 헤지펀드의 다양한 투자 전략 중 하나로 자리잡았다.버핏의 일본 종합상사 주식 보유 비중은 점차 늘어 최근에는 7.4%까지 늘어난 것으로 알려졌다.한편 버핏은 최근 지분을 축소한 중국 전기차 업체 비야디(BYD)와 대만 반도체 업체 TSMC에 대해서는 여전히 기업가치를 높게 평가하고 있다고 밝혔습니다.
[1] 이 영화가 지속적으로 어필하는 또 다른 이유는 이 영화의 역사적 중요성입니다.[3] 그는 부하들이 육체 노동 프로젝트에 참여하는 것을 거부하고 일본 수용소 사령관 사이토 대령(Sessue Hayakawa)을 화나게 합니다.콰이강의 다리는 실제 사건에 기반을 두고 있으며, 오늘날에도 여전히 관련이 있는 역사의 시기를 엿볼 수 있습니다.그것은 전쟁의 참혹함과 과거의 실수로부터 배우는 것의 중요성을 상기시켜 주는 역할을 합니다.[2] 콰이 강의 다리는 엄청난 역경 속에서도 용기와 결단의 서사시다.그것은 전쟁의 공포와 회복력과 지구력을 일깨워주는 

In [13]:
# LoRA에서 사용하는 low-rank matrices 어텐션 차원을 정의. 여기서는 64로 설정
# 값이 크면 클수록 더 많은 수정이 이루어지며, 모델이 더 복잡해질 수 있음
lora_r = 64   

# LoRA 적용 시 가중치에 곱해지는 스케일링 요소. 여기서는 16으로 설정
# LoRA가 적용될 때 원래 모델의 가중치에 얼마나 영향을 미칠지 결정. 높은 값은 가중치 조정의 강도를 증가시킴
lora_alpha = 16  

# Dropout probability for LoRA layers   # LoRA 층에 적용되는 드롭아웃 확률. 여기서는 0.1 (10%)로 설정
lora_dropout = 0.1 # 일부 네트워크 연결을 무작위로 비활성화하여 모델의 강건함에 기여

In [14]:
#모델이 예측한 결과와 체크포인트가 저장될 출력 디렉터리
output_dir = "./results" 

# 훈련 에포크 수
num_train_epochs = 2

# fp16/bf16 학습 활성화(A100으로 bf16을 True로 설정)
fp16 = False   
bf16 = False   

# 훈련용 배치 크기
per_device_train_batch_size = 2

# 평가용 배치 크기
per_device_eval_batch_size = 1  

# 그래디언트를 누적할 업데이트 스텝 횟수
gradient_accumulation_steps = 1  

# 그래디언트 체크포인트 활성화
gradient_checkpointing = True  


# 그래디언트 클리핑을 위한 최대 그래디언트 노름을 설정. 
# 그래디언트 클리핑은 그래디언트의 크기를 제한하여 훈련 중 안정성을 높임.
# Maximum gradient normal (그래디언트 클리핑) 0.3으로 설정
max_grad_norm = 0.3  

# 초기 학습률 AdamW 옵티마이저
learning_rate = 2e-6 

# bias/LayerNorm 가중치를 제외하고 모든 레이어에 적용할 Weight decay 값
weight_decay = 0.001 

# 옵티마이저 설정
optim = "paged_adamw_32bit"  

# 학습률 스케줄러의 유형 설정, 여기서는 코사인 스케줄러 사용
lr_scheduler_type = "cosine"   

# 훈련 스텝 수(num_train_epochs 재정의)
max_steps = -1 

# (0부터 learning rate까지) 학습 초기에 학습률을 점진적으로 증가시키 linear warmup 스텝의 Ratio
warmup_ratio = 0.03  

# 시퀀스를 동일한 길이의 배치로 그룹화, 메모리 절약 및 훈련 속도를 높임
group_by_length = True   

# X 업데이트 단계마다 체크포인트 저장
save_steps = 0  

# 매 X 업데이트 스텝 로그
logging_steps = 25  

In [15]:
# 최대 시퀀스 길이 설정
max_seq_length = None 

# 동일한 입력 시퀀스에 여러 개의 짧은 예제를 넣어 효율성을 높일 수 있음
packing = False  

# GPU 0 전체 모델 로드 
device_map = {"": 0}  

In [16]:
# 4-bit precision 기반의 모델 로드
use_4bit = True 

# 4비트 기반 모델에 대한 dtype 계산
bnb_4bit_compute_dtype = "float16" 

# 양자화 유형(fp4 또는 nf4)
bnb_4bit_quant_type = "nf4" 

# 4비트 기 모델에 대해 중첩 양자화 활성화(이중 양자화)
use_nested_quant = False 

In [17]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
# 모델 계산에 사용될 데이터 타입 결정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,  # 모델을 4비트로 로드할지 여부를 결정
    bnb_4bit_quant_type=bnb_4bit_quant_type, # 양자화 유형을 설정
    bnb_4bit_compute_dtype=compute_dtype,  # 계산에 사용될 데이터 타입을 설정
    bnb_4bit_use_double_quant=use_nested_quant, # 중첩 양자화를 사용할지 여부를 결정
)

In [18]:
# 만약 GPU가 최소한 버전 8 이상이라면 (major >= 8) bfloat16을 지원한다고 메시지를 출력. 
# bfloat16은 훈련 속도를 높일 수 있는 데이터 타입. 

if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)
    else:
        print("=" * 80)
        print("Your GPU don\'t supports bfloat16: accelerate training with bf16=False")
        print("=" * 80)

Your GPU don't supports bfloat16: accelerate training with bf16=False


In [19]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

In [20]:
from peft import get_peft_model
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM", # 파인튜닝할 태스크를 Optional로 지정할 수 있는데, 여기서는 CASUAL_LM을 지정하였다.
)
model = get_peft_model(base_model, peft_config)
model.print_trainable_parameters()

trainable params: 52,428,800 || all params: 14,550,717,440 || trainable%: 0.3603


In [21]:
# Trainer with LoRA configuration
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset_training_data,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=base_tokenizer,
    args=training_arguments,
    packing=packing,
)

/home/dxlab/anaconda3/envs/obqa/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/dxlab/anaconda3/envs/obqa/lib/python3.11/site-packages/transformers/training_args.py:1815: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/home/dxlab/anaconda3/envs/obqa/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:278: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/home/dxlab/anaconda3/envs/obqa/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

/home/dxlab/anaconda3/envs/obqa/lib/python3.11/site-packages/accelerate/accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [22]:
# Training
trainer.train()
# Save trained model
trainer.model.save_pretrained(new_model)

Step,Training Loss
25,2.036200
50,2.415300
75,2.419200
100,2.166900
125,2.376900
150,2.317800


/home/dxlab/anaconda3/envs/obqa/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [23]:
# base_model과 new_model에 저장된 LoRA 가중치를 통합하여 새로운 모델을 생성
base_model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(base_model, new_model) # LoRA 가중치를 가져와 기본 모델에 통합

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [24]:
model = model.merge_and_unload()

In [28]:
torch.cuda.empty_cache()

# Generate text using fine-tuned model
context = '	열사병을 예방하는 방법 겨울보다 여름철 유독 많이 발생하는 열사병을 예방하기 위해서는 고온 다습하고 더운 환경에 노출하지 않는 게 좋습니다. 강한 자외선과 더운 날씨에 외출을 가급적 삼가를 해야 하며 몸으로 빠져나가는 수분을 이온음료, 물 등을 수시로 섭취하여 보충해줘야 합니다. 심할 경우 사망에까지 이를 수 있으므로 각별히 주의해야 하는데요, 응급처치로는 환자를 그늘지고 시원한 곳으로 옮긴 후 옷을 느슨하게 풀어주고 물수건으로 닦아주는 방법이 있습니다. 특히 온열질환의 초기 증상은 대체적으로 약한 두통과 어지러움 그리고 속이 울렁거린다는 공통점이 있는데요 이러한 증상을 시작으로 점점 심화되기 때문에 초기에 나타나는 증상을 가급적 빨리 감지하고 조치를 하면 좋습니다. 그런데 요즘은 더위가 빨리 찾아오고, 불가피하게 폭염특보 때문에 외출을 하는 사람들이 많아 이런 병에 걸리는 경우가 많았습니다. 따라서 부득이하게 야외에 나갈 때는 챙이 넓은 모자를 쓰고 헐렁한 옷을 입는 것이 좋습니다. 또한, 규칙적으로 물을 마시고 몸에 수분이 부족하지 않도록 주의하는 것도 좋습니다. 무엇보다 일사병 증상 중 정신이상이 나타나는 경우 열사병으로 의심해 보아야 하기 때문에 반드시 병원 진료를 받아야합니다. 일사병은 대체적으로 염분과 수분의 부족으로 나타나는 경우가 많으므로 규칙적인 수분 섭취를 통해 예방할 수 있습니다. 일시적으로 일사병의 증상이 나타나는 경우 서늘한 곳에서 휴식을 취하고 수분을 섭취하면 호전되는 경우가 많습니다. 무엇보다 일사병 증상 중 정신이상이 나타나는 경우 열사병으로 의심해 보아야 하기 때문에 반드시 병원 진료를 받아야합니다. 일사병은 대체적으로 염분과 수분의 부족으로 나타나는 경우가 많으므로 규칙적인 수분 섭취를 통해 예방할 수 있습니다. 일시적으로 일사병의 증상이 나타나는 경우 서늘한 곳에서 휴식을 취하고 수분을 섭취하면 호전되는 경우가 많습니다.'
question = '여름철에 많이 생기는 열사병을 예방하는 방법은 무엇인가요?'
query =  f'<|begin_of_text|><|start_header_id|>context<|end_header_id|>\n\n{context}<|eot_id|><|start_header_id|>question<|end_header_id|>\n\n{question}<|eot_id|><|start_header_id|>answer<|end_header_id|>\n\n'
query_4 = f'<|begin_of_text|><|start_header_id|>context<|end_header_id|>\n\n{context}<|eot_id|><|start_header_id|>question<|end_header_id|>\n\n{question}<|eot_id|>'

ex_question =  "워런 버핏이 제안하는 투자 전략은 무엇인가요?"
ex_context = "[1] 이어 최근 주식 보유 비중을 높인 일본 종합상사에 대해서는 “앞으로 100년 동안, 아니 영원히 살아남을 기업”이라고 말했다. 저평가된 우량주를 매입해 장기 보유하는 가치 투자 방식을 고집하는 버핏은 2020년 8월 이후부터 이토추상사, 마루베니, 미쓰비시상사, 미쓰이물산, 스미토모상사 등 일본 5대 종합상사의 주식을 사들여왔다. 버핏의 일본 종합상사 주식 보유 비중은 점차 늘어 최근에는 7.4%까지 늘어난 것으로 알려졌다. [2] 적극적으로 행사해 그 차이를 빠르게 줄이는 전략이다. 그레이엄의 수제자인 워런 버핏도 이 같은 행동주의 전략을 적극 활용했다. 이후 행동주의는 1980~1990년대 ‘기업 사냥꾼 시대’를 거쳐 2000년대 들어 헤지펀드의 다양한 투자 전략 중 하나로 자리잡았다. [3] 그는 내년에 은행 파산으로 예금 손실을 보는 미국인은 없을 것이란 데 100만달러를 건다면서 반대의 경우에 돈을 거는 사람이 있다면 지는 사람이 200만달러를 자선단체에 기부하자는 제안도 했습니다. 한편 버핏은 최근 지분을 축소한 중국 전기차 업체 비야디(BYD)와 대만 반도체 업체 TSMC에 대해서는 여전히 기업가치를 높게 평가하고 있다고 밝혔습니다." 
ex_answer =  "워렌 버핏은 주식 투자에서 가치 투자 전략을 제안합니다. 이는 저평가된 우량주를 장기간 보유하는 방식으로, 일본 종합상사들의 주식을 매입해 이를 실천하고 있습니다. 이 전략은 장기적인 시각으로 기업의 가치를 평가하고, 저렴하게 평가된 주식을 구매하여 해당 기업이 성장하고 가치를 높일 때까지 보유하는 것을 목표로 합니다."


context_2 = """
열사병을 예방하는 방법 겨울보다 여름철 유독 많이 발생하는 열사병을 예방하기 위해서는 고온 다습하고 더운 환경에 노출하지 않는 게 좋습니다. 강한 자외선과 더운 날씨에 외출을 가급적 삼가를 해야 하며 몸으로 빠져나가는 수분을 이온음료, 물 등을 수시로 섭취하여 보충해줘야 합니다.
심할 경우 사망에까지 이를 수 있으므로 각별히 주의해야 하는데요, 응급처치로는 환자를 그늘지고 시원한 곳으로 옮긴 후 옷을 느슨하게 풀어주고 물수건으로 닦아주는 방법이 있습니다. 특히 온열질환의 초기 증상은 대체적으로 약한 두통과 어지러움 그리고 속이 울렁거린다는 공통점이 있는데요 이러한 증상을 시작으로 점점 심화되기 때문에 초기에 나타나는 증상을 가급적 빨리 감지하고 조치를 하면 좋습니다.
그런데 요즘은 더위가 빨리 찾아오고, 불가피하게 폭염특보 때문에 외출을 하는 사람들이 많아 이런 병에 걸리는 경우가 많았습니다. 따라서 부득이하게 야외에 나갈 때는 챙이 넓은 모자를 쓰고 헐렁한 옷을 입는 것이 좋습니다. 또한, 규칙적으로 물을 마시고 몸에 수분이 부족하지 않도록 주의하는 것도 좋습니다.
무엇보다 일사병 증상 중 정신이상이 나타나는 경우 열사병으로 의심해 보아야 하기 때문에 반드시 병원 진료를 받아야합니다. 일사병은 대체적으로 염분과 수분의 부족으로 나타나는 경우가 많으므로 규칙적인 수분 섭취를 통해 예방할 수 있습니다. 일시적으로 일사병의 증상이 나타나는 경우 서늘한 곳에서 휴식을 취하고 수분을 섭취하면 호전되는 경우가 많습니다.
"""
query_2 = f'context: {context_2} question:{question} context의 내용을 근거로 question의 내용에 답변알려줘'
query_3 = f'{context_2}{question}'

message = [
    {"role": "system", "content": query_3}
]

poly_context = run_example(ex_context, ex_question)

message_2 = [
    {"role": "context", "content": poly_context},
    {"role": "question", "content": ex_question}
]

text_gen = pipeline(
    task="text-generation",
    model=new_model, 
    tokenizer=base_tokenizer,
    max_new_tokens=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
text_gen(message_2)

Context sentences: ['[1] 이어 최근 주식 보유 비중을 높인 일본 종합상사에 대해서는 “앞으로 100년 동안, 아니 영원히 살아남을 기업”이라고 말했다.', '저평가된 우량주를 매입해 장기 보유하는 가치 투자 방식을 고집하는 버핏은 2020년 8월 이후부터 이토추상사, 마루베니, 미쓰비시상사, 미쓰이물산, 스미토모상사 등 일본 5대 종합상사의 주식을 사들여왔다.', '버핏의 일본 종합상사 주식 보유 비중은 점차 늘어 최근에는 7.4%까지 늘어난 것으로 알려졌다.', '[2] 적극적으로 행사해 그 차이를 빠르게 줄이는 전략이다.', '그레이엄의 수제자인 워런 버핏도 이 같은 행동주의 전략을 적극 활용했다.', '이후 행동주의는 1980~1990년대 ‘기업 사냥꾼 시대’를 거쳐 2000년대 들어 헤지펀드의 다양한 투자 전략 중 하나로 자리잡았다.', '[3] 그는 내년에 은행 파산으로 예금 손실을 보는 미국인은 없을 것이란 데 100만달러를 건다면서 반대의 경우에 돈을 거는 사람이 있다면 지는 사람이 200만달러를 자선단체에 기부하자는 제안도 했습니다.', '한편 버핏은 최근 지분을 축소한 중국 전기차 업체 비야디(BYD)와 대만 반도체 업체 TSMC에 대해서는 여전히 기업가치를 높게 평가하고 있다고 밝혔습니다.']
Context input IDs shape: torch.Size([8, 66])
Question input IDs shape: torch.Size([1, 15])
Number of context sentences: 8
Scores shape: torch.Size([8, 16])
Scores: tensor([[1665.2512, 1665.2512, 1665.2512, 1665.2512, 1665.2512, 1665.2512,
         1665.2512, 1665.2512, 1665.2512, 1665.2512, 1665.2512, 1665.2512,
         1665.2512, 1665.2512, 1665.2512, 16

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

[{'generated_text': [{'role': 'context',
    'content': '[2] 적극적으로 행사해 그 차이를 빠르게 줄이는 전략이다.이후 행동주의는 1980~1990년대 ‘기업 사냥꾼 시대’를 거쳐 2000년대 들어 헤지펀드의 다양한 투자 전략 중 하나로 자리잡았다.그레이엄의 수제자인 워런 버핏도 이 같은 행동주의 전략을 적극 활용했다.버핏의 일본 종합상사 주식 보유 비중은 점차 늘어 최근에는 7.4%까지 늘어난 것으로 알려졌다.한편 버핏은 최근 지분을 축소한 중국 전기차 업체 비야디(BYD)와 대만 반도체 업체 TSMC에 대해서는 여전히 기업가치를 높게 평가하고 있다고 밝혔습니다.'},
   {'role': 'question', 'content': '워런 버핏이 제안하는 투자 전략은 무엇인가요?'},
   {'role': 'assistant',
    'content': '“나는 투자는 기업의 일부를 사는 것이라고 생각한다.그리고 나는 기업의 일부를 살 때 그 회사를 잘 이해하려고 노력한다.”(워런 버핏 버크셔해서웨이 회장)버핏 회장은 ‘투자 대가’ 답게 주식 투자의 본질에 대해 명확하게 정의하고 있습니다.즉 기업을 살 때 해당 기업을 제대로 파악하고 주식을 매입해야 한다는 겁니다.주식 투자를 하는 사람은 누구나 ‘장기투자’를 하고 싶어합니다.하지만 기업의 가치에 대해 제대로 파악하지 못한 채 투자를 하는 것은 장기투자라고 말할 수 없습니다.버핏 회장은 “장기 투자는 단순히 오래 보유하기 위해서 하는 것이 아니라 기업의 가치를 제대로 파악한 후 기업의 일부를 사서 오랫동안 보유하는 것”이라며 “투자 기간이 길수록 기업의 가치가 높아지고, 장기 투자에 성공할 확률이 높아진다”고 주장합니다.버핏 회장은 기업의'}]}]